### Gemini function calling

Inspired by:
- Gen AI startup skills lab: week 3
- [Gemini function calling documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling)

In [17]:
import os

In [18]:
project_id = os.getenv('PROJECT_ID')
region_id = os.getenv('REGION_ID')
llm_model_id = os.getenv('CHAT_MODEL')
embedding_model_id = os.getenv('EMB_MODEL')

In [19]:
!pip install --upgrade --user --quiet google-cloud-aiplatform

In [20]:
import vertexai

vertexai.init(project=project_id, location=region_id)

In [21]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

### Example 1. Chat: Using Function Calling in a chat session to answer user's questions about the Google Store

#### Define three functions:
- one to get product information
- another to get the location of the closest stores
- one more to place an order

Function parameters specified as a Python dictionary in accordance with the [OpenAPI JSON schema format](https://spec.openapis.org/oas/v3.0.3#schemawr)

In [22]:
get_product_info = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        'type': 'object',
        'properties': {
            'product_name': {
                'type': 'string',
                'description': 'Product name'
            }
        }
    }
)

In [23]:
get_store_location = FunctionDeclaration(
    name='get_store_location',
    description='Get the location of the nearest store',
    parameters={
        'type': 'object',
        'properties': {
            'location': {
                'type': 'string',
                'description': "Location"
            }
        }
    }
)

In [24]:
place_order = FunctionDeclaration(
    name='place_order',
    description='Place an order',
    parameters={
        'type': 'object',
        'properties': {
            'product': {
                'type': 'string',
                'description': 'Product name'
            },
            'address': {
                'type': 'string',
                'description': 'Shipping address'
            }
        }
    }
)

#### Define a tool that allows the Gemini model to select from the set of 3 functions
- a collection of functions that the model may use to generate a response

In [25]:
retail_tools = Tool(
    function_declarations=[
        get_product_info, get_store_location, place_order]
)

#### Initialize the Gemini model with Function Calling in a multi-turn chat session

In [26]:
chat_model = GenerativeModel(
    model_name=llm_model_id,
    generation_config=GenerationConfig(temperature=0),
    tools=[retail_tools]
)
chat = chat_model.start_chat()

#### Start chat

In [29]:
question = """
Do you have the iPhone 15 Pro in stock?
"""

#### Define the user's prompt in a Content object that we can reuse in model calls

In [61]:
user_question_content = Content(
    role="user",
    parts=[
        Part.from_text(question),
    ],
)

In [62]:
user_question_content

role: "user"
parts {
  text: "\nDo you have the iPhone 15 Pro in stock?\n"
}

#### Send the user question to the model
- Note the model returns `function_call` `name: get_product_info` in the response, indicating a function of that name has to be called
- You are next supposed to call the function and get a response from that function...
- Then you send the result of that function call to the model in a subsequent message which has `name = get_product_info` with structure:
```
parts = [
        Part.from_function_response(
            name="get_product_info",
            response={
                "content": <response_from_the_function>
            }
        )
]
```

In [69]:
response = chat.send_message(user_question_content)
response_function_call_content = response.candidates[0].content
response_function_call_content

role: "model"
parts {
  function_call {
    name: "get_product_info"
    args {
      fields {
        key: "product_name"
        value {
          string_value: "iPhone 15 Pro"
        }
      }
    }
  }
}

#### Make the function call Gemini asked us to
- Note: here we just assume we call it and get a dummy response back

In [70]:
# assume get_product_info returned this dummy response
get_product_info_api_response = {"sku": "GA04834-US", "in_stock": "yes"}

#### Send the model the response from function call
- See the candidate text part for the model's response
- Note: `send_message` turn is to say this is the response for the `get_product_info` function that we were asked to call
- Note: see the `finish_reason: STOP` in the response

In [72]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_info",
        response={
            "content": get_product_info_api_response,
        },
    ),
)
print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "Yes, the iPhone 15 Pro is in stock. The SKU is GA04834-US.\n\n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0849471
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.178239584
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.163045257
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.194980219
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0706877932
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0294794086
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0302718692
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0177122988
  }
}
usage_metadata {
  prompt_token_c

#### User might ask where they can buy a different phone from a nearby store
- This starts a new turn..
- Note: `get_store_location` function_call has to be called now

In [75]:
prompt = """
What about the Pixel 8? Is there a store in
Mountain View, CA that I can visit to try one out?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

#### Make the function call Gemini asked us to

In [76]:
# dummy response for get_store_location
get_store_location_api_response = { 
    'location': '2000 N Shoreline Blvd, Mountain View, CA 94043, US'
}

#### Pass the function response from the (mock) API request back to the Gemini model
- Gemini will enforce the turn-by-turn. For eg. that Part.from_function_response is sent to the model after a function call
- Note: `send_message` sends a `Part` saying the `get_store_location` function's response is what we're sending
- Note: `finish_reason: STOP` in the response payload, indicating we should stop

In [77]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content": get_store_location_api_response,
        },
    ),
)
print(response)
# print(response.candidates[0].content.parts[0])

candidates {
  content {
    role: "model"
    parts {
      text: "Yes, there is a Google Store in Mountain View, CA that you can visit to try out the Pixel 8. The address is 2000 N Shoreline Blvd, Mountain View, CA 94043, US. "
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0758581758
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.104660198
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.233356759
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.122418255
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0905744731
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0327131264
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0532062054
    severi

#### Finally, the user might ask to order a phone and have it shipped to their address
- Note: function_call `place_order` has to be called

In [78]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
print(response)

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "place_order"
        args {
          fields {
            key: "product"
            value {
              string_value: "Pixel 8 Pro"
            }
          }
          fields {
            key: "address"
            value {
              string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.148047194
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.102664009
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.400046021
    severity: HARM_SEVERITY_LOW
    severity_score: 0.214197889
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.119819485
    severity: HARM_SE

#### Make the function call Gemini asked us to
- Note: here we just assume we call it and get a dummy response back

In [79]:
# dummy api response of `place_order`
place_order_dummy_api_response = {
    "payment_status": "paid",
    "order_number": 12345,
    "est_arrival": "2 days",
}

#### Pass the function response from the (mock) API request back to the Gemini model
- Gemini will enforce the turn-by-turn. For eg. that Part.from_function_response is sent to the model after a function call
- Note: `send_message` sends a `Part` saying the `place_order` function's response is what we're sending
- Note: `finish_reason: STOP` in the response payload, indicating we should stop

In [80]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content": place_order_dummy_api_response,
        },
    ),
)
print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "OK. I have placed your order for a Pixel 8 Pro. It will be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089. The estimated arrival time is 2 days. Your order number is 12345. The payment has been processed. \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0941766649
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0899330154
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.218169227
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.118793398
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.142828658
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0532062054
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGI

### Example 2. Chat: Using Function Calling to geocode addresses with a maps API

In [81]:
get_location = FunctionDeclaration(
    name="get_location",
    description="Get latitude and longitude for a given location",
    parameters={
        "type": "object",
        "properties": {
            "poi": {"type": "string", "description": "Point of interest"},
            "street": {"type": "string", "description": "Street name"},
            "city": {"type": "string", "description": "City name"},
            "county": {"type": "string", "description": "County name"},
            "state": {"type": "string", "description": "State name"},
            "country": {"type": "string", "description": "Country name"},
            "postal_code": {"type": "string", "description": "Postal code"},
        },
    },
)

location_tool = Tool(
    function_declarations=[get_location],
)

In [83]:
prompt = """
I want to get the coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043, US
"""

response = chat_model.generate_content(
    prompt,
    generation_config=GenerationConfig(temperature=0),
    tools=[location_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_location"
        args {
          fields {
            key: "street"
            value {
              string_value: "1600 Amphitheatre Pkwy"
            }
          }
          fields {
            key: "state"
            value {
              string_value: "CA"
            }
          }
          fields {
            key: "postal_code"
            value {
              string_value: "94043"
            }
          }
          fields {
            key: "country"
            value {
              string_value: "US"
            }
          }
          fields {
            key: "city"
            value {
              string_value: "Mountain View"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0841910839
    severity: HARM_SEVERITY_NEGLIGIBLE
    severi

#### reference the parameters from the function call and make a live API request

In [84]:
x = response.candidates[0].content.parts[0].function_call.args

url = "https://nominatim.openstreetmap.org/search?"
for i in x:
    url += '{}="{}"&'.format(i, x[i])
url += "format=json"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
x = requests.get(url, headers=headers)
content = x.json()
content

# Note: if you get a JSONDecodeError when running this cell, try modifying the
# user agent string in the `headers=` line of code in this cell and re-run.

[{'place_id': 377680635,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 2192620021,
  'lat': '37.4217636',
  'lon': '-122.084614',
  'class': 'office',
  'type': 'it',
  'place_rank': 30,
  'importance': 0.6949356759210291,
  'addresstype': 'office',
  'name': 'Google Headquarters',
  'display_name': 'Google Headquarters, 1600, Amphitheatre Parkway, Mountain View, Santa Clara County, California, 94043, United States',
  'boundingbox': ['37.4217136', '37.4218136', '-122.0846640', '-122.0845640']}]

### Example 3. Logging example: Using Function Calling for entity extraction only

In [85]:
extract_log_data = FunctionDeclaration(
    name="extract_log_data",
    description="Extract details from error messages in raw log data",
    parameters={
        "type": "object",
        "properties": {
            "locations": {
                "type": "array",
                "description": "Errors",
                "items": {
                    "description": "Details of the error",
                    "type": "object",
                    "properties": {
                        "error_message": {
                            "type": "string",
                            "description": "Full error message",
                        },
                        "error_code": {"type": "string", "description": "Error code"},
                        "error_type": {"type": "string", "description": "Error type"},
                    },
                },
            }
        },
    },
)

#### define a tool for the generative model to call that includes the `extract_log_data`

In [86]:
extraction_tool = Tool(
    function_declarations=[extract_log_data],
)

#### pass the sample log data to the Gemini model
- The model will call the log extractor function, and the model output will be a Function Call response.
- response includes a structured data object that contains the details of the error messages that appear in the log.

In [87]:
prompt = """
[15:43:28] ERROR: Could not process image upload: Unsupported file format. (Error Code: 308)
[15:44:10] INFO: Search index updated successfully.
[15:45:02] ERROR: Service dependency unavailable (payment gateway). Retrying... (Error Code: 5522)
[15:45:33] ERROR: Application crashed due to out-of-memory exception. (Error Code: 9001)
"""

response = chat_model.generate_content(
    prompt,
    generation_config=GenerationConfig(temperature=0),
    tools=[extraction_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "extract_log_data"
        args {
          fields {
            key: "locations"
            value {
              list_value {
                values {
                  struct_value {
                    fields {
                      key: "error_type"
                      value {
                        string_value: "ERROR"
                      }
                    }
                    fields {
                      key: "error_message"
                      value {
                        string_value: "Could not process image upload: Unsupported file format."
                      }
                    }
                    fields {
                      key: "error_code"
                      value {
                        string_value: "308"
                      }
                    }
                  }
                }
                values {
                  struct_value {
 